In [1]:
import pandas as pd
import os
import numpy as np
from lexicalrichness import LexicalRichness
import textstat
import re
import nltk
from collections import Counter
from scipy.stats import ttest_ind
from scipy.stats import levene
from scipy.stats import wald
from scipy.stats import mannwhitneyu
import itertools

In [2]:
inpath2= os.path.dirname(os.path.dirname(os.getcwd()))+"\\Data\\Intermediate Data\\"
df=pd.read_csv(inpath2+'final_all_files.csv')

In [3]:
df['text_len']=df['text'].apply(lambda x: len(x))

In [4]:
df['mattr'] = df['text'].apply(lambda x: LexicalRichness(x).mattr(window_size=25))

In [5]:
df['FK']=df['text'].apply(lambda x: textstat.flesch_kincaid_grade(x))

In [6]:
def tokenize(data):
    return nltk.word_tokenize(data)
def get_counts(tokens):
    counts=Counter()
    for token in tokens:
        counts[token]+=1
    return counts

In [7]:
def calculate_tc(testText): 
    tokens=tokenize(testText)
    token_counts=get_counts(tokens)
    freq_df=pd.DataFrame.from_dict(token_counts, orient='index', columns=['frequency']).reset_index().sort_values(by='frequency', ascending=False)
    freq_df['x']=1
    freq_df['rank']=freq_df.groupby('x').cumcount()+1
    fr1=freq_df.iloc[0,1]
    fr2=freq_df.iloc[1,1]
    r1=freq_df.iloc[0,3]
    r2=freq_df.iloc[1,3]
    h_point=((fr1*r2)-(fr2*r1))/(r2-r1+fr1-fr2)
    freq_df['TW']=np.where(freq_df['rank']<h_point, 2*(((h_point-freq_df['rank'])*(freq_df['frequency']))/((h_point*(h_point-1)*fr1))), 0)
    return freq_df['TW'].sum()

In [8]:
df['TC']=df['text'].apply(lambda x: calculate_tc(x))

In [9]:
trump=df[df['candidate']=='trump']

In [10]:
trump['rally']=np.where(trump['event_type']=='rally', 1, 0)

C:\Users\pcrrt\AppData\Local\Temp/ipykernel_20028/3625745240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trump['rally']=np.where(trump['event_type']=='rally', 1, 0)


In [11]:
trump_rallies=trump[trump['rally']==1]
trump_not_rallies=trump[trump['rally']==0]

In [12]:
ttest_ind(trump_rallies.TC, trump_not_rallies.TC, equal_var = False)

Ttest_indResult(statistic=-1.1226434765155002, pvalue=0.2627954556521124)

In [13]:
mannwhitneyu(trump_rallies.TC, trump_not_rallies.TC)

MannwhitneyuResult(statistic=5446.5, pvalue=0.028287041805636834)

In [14]:
ttest_ind(trump_rallies.FK, trump_not_rallies.FK, equal_var = False)

Ttest_indResult(statistic=0.5213342215547009, pvalue=0.602683910198359)

In [15]:
mannwhitneyu(trump_rallies.FK, trump_not_rallies.FK)

MannwhitneyuResult(statistic=3184.0, pvalue=3.67446677801599e-11)

In [16]:
ttest_ind(trump_rallies.mattr, trump_not_rallies.mattr, equal_var = False)

Ttest_indResult(statistic=-13.357719864481169, pvalue=2.606497694844693e-29)

In [17]:
mannwhitneyu(trump_rallies.mattr, trump_not_rallies.mattr)

MannwhitneyuResult(statistic=1110.5, pvalue=3.7043903298640934e-27)

In [18]:
trump.groupby('rally')[['mattr', 'TC', 'FK']].describe()

mattr                                                              \
       count      mean       std       min       25%       50%       75%   
rally                                                                      
0      112.0  0.852888  0.018664  0.814184  0.840418  0.850485  0.864273   
1      114.0  0.824522  0.012630  0.802451  0.817030  0.823022  0.828617   

                    TC            ...                         FK              \
            max  count      mean  ...       75%       max  count        mean   
rally                             ...                                          
0      0.904532  112.0  0.692803  ...  0.799161  0.912281  112.0  270.036607   
1      0.888349  114.0  0.669341  ...  0.777531  0.892642  114.0  335.015789   

                                                
               std  min  25%  50%  75%     max  
rally                                           
0       797.267424  1.7  5.0  6.4  8.9  4190.7  
1      1060.199711  3.6  4.0  4.5  5.4  5945.3  

[2 rows x 24 columns]

In [19]:
trump.event_type.value_counts()

rally     114
speech    109
debate      3
Name: event_type, dtype: int64

In [ ]:
#only certain frequencies, stop words - LIWC - 70-80 features. Mispelled - not going to be able to find it
#topic modeling - dimensionality reduction
#most relevant words for each topic - give an identity to each topic - topic modeling to predict features for each
#5 topics - how prominent the topic is within the document = use that score as the feature - almost 30 x 5 - good row to column ratio
#For every feature - 6 rows - 
#sequence embedding for each speech - could cluster - look for certain outliers - one particular rally speech
#visualize in a two d pot
#t-ste - condense to two-d ste - interactive 2-d scatter plot - certain keywords
#pattern you can see from the sequence embeddings - color the dots - see that the red dots - 
#bert topic modeling 
#tfid - word threshold - top 1000 words - LIWC version - convert every word to LIWC features
#ngram=(2,2) - bigrams, max_df - do not only include words show up in 50% or fewer documents - 200 documents or less.
#min_df - only tokenize words that show up in at least 5 documents
#max_features - only 1000 most popular words! 
#binary - if word in the document or not
#tfid - see words that bring value - stop words - show up in many different documents
#low tfid - stopwords - are not labeled stopwords - find a way to help reduce the number of tokens - pass that into tfid
#once you fit tfidf - find the tokens that have a high tf-idf.
#hate speech/dehumanization - 
#https://towardsdatascience.com/kl-divergence-python-example-b87069e4b810
#https://towardsdatascience.com/interactive-topic-modeling-with-bertopic-1ea55e7d73d8
#https://cs.stanford.edu/people/wmorgan/sigtest.pdf